### Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from datetime import datetime as dt

### The Cases dashboard

In [2]:
casesDashboard = dash.Dash(__name__)

### Import the Cases data

In [3]:
cases = pd.read_csv('../covid_confirmed_usafacts.csv')
cases.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2023-07-14,2023-07-15,2023-07-16,2023-07-17,2023-07-18,2023-07-19,2023-07-20,2023-07-21,2023-07-22,2023-07-23
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,19913,19913,19913,19913,19913,19913,19913,19913,19913,19913
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,70521,70521,70521,70521,70521,70521,70521,70521,70521,70521
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,7582,7582,7582,7582,7582,7582,7582,7582,7582,7582
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,8149,8149,8149,8149,8149,8149,8149,8149,8149,8149


### Clean and Format the Cases dataframe

In [4]:
selected_date_columns = [col for col in cases.columns if '2020-01-22' <= col <= '2023-07-23']
cases = cases[selected_date_columns]
cases.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2023-07-14,2023-07-15,2023-07-16,2023-07-17,2023-07-18,2023-07-19,2023-07-20,2023-07-21,2023-07-22,2023-07-23
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,19913,19913,19913,19913,19913,19913,19913,19913,19913,19913
2,0,0,0,0,0,0,0,0,0,0,...,70521,70521,70521,70521,70521,70521,70521,70521,70521,70521
3,0,0,0,0,0,0,0,0,0,0,...,7582,7582,7582,7582,7582,7582,7582,7582,7582,7582
4,0,0,0,0,0,0,0,0,0,0,...,8149,8149,8149,8149,8149,8149,8149,8149,8149,8149


### Cases Dashboard layout

In [5]:
casesDashboard.layout = html.Div([

    html.H1("Interactive Dashboard for COVID-19 Cases", style={'text-align': 'center'}),

    dcc.DatePickerRange(
        id='date-picker-range',
        start_date='2020-01-22',
        end_date='2023-07-23',
        display_format='YYYY-MM-DD',
        min_date_allowed='2020-01-22',
        max_date_allowed='2023-07-23',
    ),
    
    dcc.RadioItems(
        id='y-axis-scale',
        options=[
            {'label': 'Linear Scale', 'value': 'linear'},
            {'label': 'Log Scale', 'value': 'log'},
        ],
        value='linear',
        labelStyle={'display': 'block'},
    ),
    
    dcc.Graph(id='cases-graph'),

])

In [6]:
@casesDashboard.callback(
    Output('cases-graph', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('y-axis-scale', 'value')]
)
def update_graph(start_date, end_date, y_axis_scale):
    selected_date_columns = [col for col in cases.columns if start_date <= col <= end_date]
    cases_df = cases[selected_date_columns]
    
    date_range = pd.date_range(start=start_date, end=end_date)
    
    daily_cases = []
    for col in cases.columns:
        daily_cases.append(cases[col].sum())
    daily_cases_data = pd.Series(daily_cases)
    
    days = np.arange(len(daily_cases_data))
    
    poly = PolynomialFeatures(degree=4)
    X_poly = poly.fit_transform(days.reshape(-1, 1))

    pr_cases = LinearRegression()
    pr_cases.fit(X_poly, daily_cases_data)
    cases_poly_predictions = pr_cases.predict(X_poly)

    if y_axis_scale == 'log':
        figure = {
        'data': [
            {'x': list(range(len(date_range))), 'y': np.log(cases_df.sum()), 'type': 'line', 'name': 'Cases'},
            {'x': list(range(len(date_range))), 'y': np.log(cases_poly_predictions), 'type': 'line', 'name': 'Predicted Cases', 'line': {'dash': 'dash'}}
            
        ],
        'layout': {
            'title': f'COVID-19 Cases from {start_date} to {end_date}',
            'xaxis': {'title': 'Days'},
            'yaxis': {'title': 'Number of Cases'}
            }
        }
    else:
        figure = {
        'data': [
            {'x': list(range(len(date_range))), 'y': cases_df.sum(), 'type': 'line', 'name': 'Cases'},
            {'x': list(range(len(date_range))), 'y': cases_poly_predictions, 'type': 'line', 'name': 'Predicted Cases', 'line': {'dash': 'dash'}}
            
        ],
        'layout': {
            'title': f'COVID-19 Cases from {start_date} to {end_date}',
            'xaxis': {'title': 'Days'},
            'yaxis': {'title': 'Number of Cases'}
            }
        }
        

    return figure

### The Deaths dashboard

In [7]:
deathsDashboard = dash.Dash(__name__)

### Import the Deaths data

In [8]:
deaths = pd.read_csv('../covid_deaths_usafacts.csv')
deaths.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2023-07-14,2023-07-15,2023-07-16,2023-07-17,2023-07-18,2023-07-19,2023-07-20,2023-07-21,2023-07-22,2023-07-23
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,235,235,235,235,235,235,235,235,235,235
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,731,731,731,731,731,731,731,731,731,731
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,104,104,104,104,104,104,104,104,104,104
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,111,111,111,111,111,111,111,111,111,111


### Clean and Format the Deaths dataframe

In [9]:
selected_date_columns = [col for col in deaths.columns if '2020-01-22' <= col <= '2023-07-23']
deaths = deaths[selected_date_columns]
deaths.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2023-07-14,2023-07-15,2023-07-16,2023-07-17,2023-07-18,2023-07-19,2023-07-20,2023-07-21,2023-07-22,2023-07-23
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,235,235,235,235,235,235,235,235,235,235
2,0,0,0,0,0,0,0,0,0,0,...,731,731,731,731,731,731,731,731,731,731
3,0,0,0,0,0,0,0,0,0,0,...,104,104,104,104,104,104,104,104,104,104
4,0,0,0,0,0,0,0,0,0,0,...,111,111,111,111,111,111,111,111,111,111


### Deaths Dashboard layout

In [10]:
deathsDashboard.layout = html.Div([

    html.H1("Interactive Dashboard for COVID-19 Deaths", style={'text-align': 'center'}),

    dcc.DatePickerRange(
        id='date-picker-range',
        start_date='2020-01-22',
        end_date='2023-07-23',
        display_format='YYYY-MM-DD',
        min_date_allowed='2020-01-22',
        max_date_allowed='2023-07-23',
    ),
    
    dcc.RadioItems(
        id='y-axis-scale',
        options=[
            {'label': 'Linear Scale', 'value': 'linear'},
            {'label': 'Log Scale', 'value': 'log'},
        ],
        value='linear',
        labelStyle={'display': 'block'},
    ),
    
    dcc.Graph(id='deaths-graph'),

])

In [11]:
@deathsDashboard.callback(
    Output('deaths-graph', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('y-axis-scale', 'value')]
)
def update_graph(start_date, end_date, y_axis_scale):
    selected_date_columns = [col for col in cases.columns if start_date <= col <= end_date]
    
    deaths_df = deaths[selected_date_columns]
    
    date_range = pd.date_range(start=start_date, end=end_date)
    
    daily_deaths = []
    for col in deaths.columns:
        daily_deaths.append(deaths[col].sum())
    daily_deaths_data = pd.Series(daily_deaths)
    
    days = np.arange(len(daily_deaths_data))
    
    poly = PolynomialFeatures(degree=4)
    X_poly = poly.fit_transform(days.reshape(-1, 1))
    
    pr_deaths = LinearRegression()
    pr_deaths.fit(X_poly, daily_deaths_data)
    deaths_poly_predictions = pr_deaths.predict(X_poly)

    if y_axis_scale == 'log':
        figure = {
        'data': [
            {'x': list(range(len(date_range))), 'y': np.log(deaths_df.sum()), 'type': 'line', 'name': 'Deaths'},
            {'x': list(range(len(date_range))), 'y': np.log(deaths_poly_predictions), 'type': 'line', 'name': 'Predicted Deaths', 'line': {'dash': 'dash'}}
            
        ],
        'layout': {
            'title': f'COVID-19 Deaths from {start_date} to {end_date}',
            'xaxis': {'title': 'Days'},
            'yaxis': {'title': 'Number of Deaths'},
            }
        }
    else:
        figure = {
        'data': [
            {'x': list(range(len(date_range))), 'y': deaths_df.sum(), 'type': 'line', 'name': 'Deaths'},
            {'x': list(range(len(date_range))), 'y': deaths_poly_predictions, 'type': 'line', 'name': 'Predicted Deaths', 'line': {'dash': 'dash'}}
            
        ],
        'layout': {
            'title': f'COVID-19 Deaths from {start_date} to {end_date}',
            'xaxis': {'title': 'Days'},
            'yaxis': {'title': 'Number of Deaths'},
            }
        }
        

    return figure

### Display the Cases dashboard

In [12]:
casesDashboard.run_server(mode='inline', port=8050)

### Display the Deaths dashboard

In [13]:
deathsDashboard.run_server(mode='inline', port=8051)